In [14]:
import torch
import torch.nn.functional as F

In [15]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [16]:
words = open('names.txt', 'r').read().splitlines()

words[:10]

['emma',
 'olivia',
 'ava',
 'isabella',
 'sophia',
 'charlotte',
 'mia',
 'amelia',
 'harper',
 'evelyn']

In [17]:
len(words)

32033

In [18]:
chars = sorted(list(set(''.join(words))))

chars

['a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']

In [19]:
N = len(chars)

In [20]:
# Define the special character that will be used to pad the words
SPECIAL = '.'

In [21]:
# Build the vocabulary of characters and the mapping from character to index
stoi = {ch: i for i, ch in enumerate(chars)}
stoi[SPECIAL] = N

stoi

{'a': 0,
 'b': 1,
 'c': 2,
 'd': 3,
 'e': 4,
 'f': 5,
 'g': 6,
 'h': 7,
 'i': 8,
 'j': 9,
 'k': 10,
 'l': 11,
 'm': 12,
 'n': 13,
 'o': 14,
 'p': 15,
 'q': 16,
 'r': 17,
 's': 18,
 't': 19,
 'u': 20,
 'v': 21,
 'w': 22,
 'x': 23,
 'y': 24,
 'z': 25,
 '.': 26}

In [22]:
# Build the inverse mapping from index to character
itos = {i: ch for ch, i in stoi.items()}

itos

{0: 'a',
 1: 'b',
 2: 'c',
 3: 'd',
 4: 'e',
 5: 'f',
 6: 'g',
 7: 'h',
 8: 'i',
 9: 'j',
 10: 'k',
 11: 'l',
 12: 'm',
 13: 'n',
 14: 'o',
 15: 'p',
 16: 'q',
 17: 'r',
 18: 's',
 19: 't',
 20: 'u',
 21: 'v',
 22: 'w',
 23: 'x',
 24: 'y',
 25: 'z',
 26: '.'}

In [38]:
# Define the context window size for the model
# This time we are not using bigrams like in the previous notebook
# We are using a context window of 3 characters to predict the next character
# This is what we call a higher-order Markov model (that we learned in TAI 2024)
K = 3

In [34]:
# Build the dataset

def create_dataset(words, stoi, K=3):
    xs, ys = [], []
    
    # Loop through each word in the dataset
    for word in words:
        # Add special characters to the beginning and end of the word
        word = SPECIAL * K + word + SPECIAL

        # Get the length of the word
        L = len(word)

        # Slide a window of size K over the word
        for i in range(L - K):
            # Get the input and output characters
            x = word[i:i + K]
            y = word[i + K]

            # Print the input and output characters
            print(''.join(x), '->', y)

            # Convert the characters to indices and add them to the dataset
            xs.append([stoi[ch] for ch in x])
            ys.append(stoi[y])
            
    return torch.tensor(xs), torch.tensor(ys)

xs, ys = create_dataset(words[:5], stoi, K)

... -> e
..e -> m
.em -> m
emm -> a
mma -> .
... -> o
..o -> l
.ol -> i
oli -> v
liv -> i
ivi -> a
via -> .
... -> a
..a -> v
.av -> a
ava -> .
... -> i
..i -> s
.is -> a
isa -> b
sab -> e
abe -> l
bel -> l
ell -> a
lla -> .
... -> s
..s -> o
.so -> p
sop -> h
oph -> i
phi -> a
hia -> .


In [35]:
xs, ys

(tensor([[26, 26, 26],
         [26, 26,  4],
         [26,  4, 12],
         [ 4, 12, 12],
         [12, 12,  0],
         [26, 26, 26],
         [26, 26, 14],
         [26, 14, 11],
         [14, 11,  8],
         [11,  8, 21],
         [ 8, 21,  8],
         [21,  8,  0],
         [26, 26, 26],
         [26, 26,  0],
         [26,  0, 21],
         [ 0, 21,  0],
         [26, 26, 26],
         [26, 26,  8],
         [26,  8, 18],
         [ 8, 18,  0],
         [18,  0,  1],
         [ 0,  1,  4],
         [ 1,  4, 11],
         [ 4, 11, 11],
         [11, 11,  0],
         [26, 26, 26],
         [26, 26, 18],
         [26, 18, 14],
         [18, 14, 15],
         [14, 15,  7],
         [15,  7,  8],
         [ 7,  8,  0]]),
 tensor([ 4, 12, 12,  0, 26, 14, 11,  8, 21,  8,  0, 26,  0, 21,  0, 26,  8, 18,
          0,  1,  4, 11, 11,  0, 26, 18, 14, 15,  7,  8,  0, 26]))

In [37]:
xs.shape, xs.dtype, ys.shape, ys.dtype

(torch.Size([32, 3]), torch.int64, torch.Size([32]), torch.int64)

In [39]:
# Now, we will create a neural network model that takes the indices of the characters in the context window as input and predicts the index of the next character.
# First, we will build the embedding lookup table that maps the character indices to their embeddings.
# So, we have N + 1 characters (including the special character) and we will map them to an embedding of size D.
# Since we have a few number of characters, we will use a small embedding size of D = 2 for now.

# Define the number of characters and the embedding size
D = 2

# Create the embedding lookup table with N + 1 characters and D dimensions and initialize it randomly
C = torch.randn((N + 1, D))

C

tensor([[-0.7570,  2.7416],
        [ 0.1253, -1.3804],
        [-0.0276,  1.8062],
        [ 1.1212, -0.5921],
        [ 0.6326, -1.1594],
        [ 0.0355, -0.6052],
        [-2.8315,  0.0387],
        [-0.2734, -0.2890],
        [ 0.3583, -1.1933],
        [-0.4516, -2.1629],
        [-0.2730, -0.3183],
        [-0.4963, -1.1389],
        [ 1.1615,  0.2430],
        [-0.0942, -1.2870],
        [-0.9104, -0.9098],
        [-0.9621,  0.1775],
        [-1.7263, -2.1575],
        [-0.8047, -0.6147],
        [ 1.2772, -0.5029],
        [ 1.1255,  0.1676],
        [ 0.3237, -0.4461],
        [-1.0830, -0.2872],
        [-0.1616, -1.6518],
        [-0.5570, -0.4926],
        [ 1.6354,  1.1351],
        [-0.7456, -1.4554],
        [-0.6175,  2.6224]])

In [40]:
# Now, before we embed all the input characters using the embedding lookup table, let's first try to embed a single individual integer to 
# get a sense of how the embedding lookup table works.

# One way to do this is just take the lookup table and index it with the integer value.
# For example, if we want to embed the character 'a' which has an index of 0, we can just index the lookup table with 0 to get the embedding of 'a'.
C[0]

tensor([-0.7570,  2.7416])

In [ ]:
# The other way presented in the previous notebook is to use one-hot encoding to convert the integer to a one-hot vector and 
# then perform a matrix multiplication with the embedding lookup table.

# Note that we need to convert the integer to a tensor before performing the one-hot encoding, because the one_hot function only works with tensors.
# We also need to convert the one-hot vector to a float tensor before performing the matrix multiplication, because PyTorch is dumb and doesn't do this automatically.
# This will just give us the same embedding as before.
# It plucks out the row corresponding to the index of the character in the embedding lookup table.
# This tells us that we can interpret the embedding of an integer as the integer indexing into the embedding lookup table, but equivalently, 
# we can interpret it as a first layer of the neural network, with no non-linearity, that is just a linear layer with the weight matrix 
# being the embedding lookup table.
# We are just going to index because it is much faster and so we will discard that interpretation of one-hot inputs into neural networks.
F.one_hot(torch.tensor([0]), N + 1).float() @ C

In [41]:
# Now, embedding a single integer is easy enough, we can just ask the embedding lookup table for the embedding of that integer.
# But, how do we simultaneously embed all of the k-order contexts present in the dataset?
# Luckily, PyTorch is quite flexible and not only allows us to index the embedding lookup table with a single integer, but also with list or tensor of integers.
# We can even index the same row multiple times.
C[torch.tensor([0, 0, 1, 1])]

tensor([[-0.7570,  2.7416],
        [-0.7570,  2.7416],
        [ 0.1253, -1.3804],
        [ 0.1253, -1.3804]])